## Cámara
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para explicar el método electoral <code>no proporcional</code></p>

El sistema electoral vigente en Colombia se encuentra definido en la reforma constitucional del [acto legislativo 01 de 2003](http://www.secretariasenado.gov.co/senado/basedoc/acto_legislativo_01_2003.html) que define metodos de cifra repartidora y umbral para las elecciones de las dos Cámaras del congreso de la republica. 

## Agenda
1. Métodos no lineales
1. Cifra repartidora
1. Método d'Hondt
1. Umbral

In [1]:
import colorama
import pandas as pd
import pylab as plt

### 1. Métodos no lineales
---
En un sistema electoral existen métodos [no lineales](https://es.wikipedia.org/wiki/Sistema_no_lineal) para repartir los escaños de un cuerpo colegiado, los más conocidos se basan en el promedio mayor y el resto mayor; los métodos de promedio mayor son menos lineales y asignan más escaños a los promedios más altos y los métodos de resto mayor asignan más escaños a los retos más altos.

### 2. Cifra repartidora
---
Los métodos de promedio mayor se basan en la división sucesiva de los votos de cada una de las listas por una sucesión de números para producir una tabla de cocientes llamados promedios, finalmente de todos estos promedios se escogen los mayores de acuerdo a una cifra mínima o cifra repartidora.

### 3. Método d'Hondt 
---
Los métodos de promedio mayor o de cifra repartidora comúnmente utilizados son el método d'Hondt y el Método Sainte-Laguë o Webster, en el primero los divisores son los números naturales (1, 2, 3, ...) y en el segundo solo los números impares (1, 3, 5, ...), de modo que las fórmulas de los cocientes d'Hondt y de Webster en cada iteración se pueden expresar como:

\begin{equation*}
Ch_{i,j} = \frac{V_i}{S_j + 1}
\end{equation*}

\begin{equation*}
Cw_{i,j} = \frac{V_i}{2S_j + 1}
\end{equation*}


><br>**Donde:** 
>* $Ch_j$ es el coeficiente d'Hondt.
>* $Cw_j$ es el coeficiente de Webster.
>* $V_i$ es el número total de votos para la i-ésima lista.
>* $S_j$ es el número j-ésimo de escaños conseguidos por la i-ésima lista.
><br><br>


### 4. Umbral 
---
En cualquiera de los métodos para repartir curules se puede establecer un umbral o limite inferior a partir del cual una lista no acumula credenciales sin importar la cifra repartidora, en el caso de la Cámara este es la mitad del cuociente electoral que depende de los votos válidos, donde los votos válidos incluyen a la suma de los votos de cada una de las listas y los votos en blanco.

\begin{equation*}
Cuociente = \frac{V}{c}
\end{equation*}

\begin{equation*}
Umbral = \frac{V}{2c}
\end{equation*}

\begin{equation*}
V = \sum{V_i} + V_{blanco}
\end{equation*}

### <code>Ejemplo:</code> método D'Hondt
---
Cálculo de credenciales al senado para cada lista en el escrutinio final de las elecciones legislativas de 2018, realizadas el domingo 11 de marzo en Colombia.

In [2]:
# Funciones
def repartidora(validos_listas: list, max_curules: int) -> int:
    ''' Función que calcula la cifra repartidora '''
    values = []
    tabla = [
        [0 for col in range(max_curules)]
        for row in range(len(validos_listas))
    ]
    count = -1
    for votos in validos_listas:
        count += 1
        for item in range(max_curules):
            value = votos/(item+1)
            values.append(value)
            tabla[count][item] = value
    return round(sorted(values)[-max_curules-2])


def cocientes(validos_listas: list, max_curules: int) -> list:
    ''' Función que calcula las curules para cada lista proporcionalmente '''
    suma = sum(validos_listas)
    proporciones = []
    resto = []
    for votos in validos_listas:
        proporciones.append(votos*max_curules/suma)
        resto.append((votos*max_curules) % suma)
    return proporciones, resto


def distribucion_proporcional(validos_listas: list, max_curules: int) -> list:
    ''' Función que calcula las curules para cada lista proporcionalmente '''
    suma = sum(validos_listas)
    curules = []
    for votos in validos_listas:
        curules.append((votos*max_curules) // suma)
    return curules


def distribucion_promedio(validos_listas: list, cifra_repartidora: int, umbral: int) -> list:
    ''' Función que Calcula las curules para cada lista con el método máximo promedio D'Hondt '''
    curules = []
    for item in validos_listas:
        if item >= umbral:
            curules.append(int(item/cifra_repartidora))
        else:
            curules.append(0)
    return curules


def distribucion_resto(validos_listas: list, max_curules: int) -> list:
    ''' Función que Calcula las curules para cada lista con el método máximo resto '''
    proporciones, resto = cocientes(validos_listas, max_curules)
    proporcionales = distribucion_proporcional(validos_listas, max_curules)
    curules_resto = int(max_curules - sum(proporcionales))
    minimo = sorted(resto)[-curules_resto]
    curules = []
    count = -1
    for item in resto:
        count += 1
        if item >= minimo:
            curules.append(proporcionales[count]+1)
        else:
            curules.append(proporcionales[count])
    return curules


In [3]:
# Cargando datos
circunscripcion = "Nariño"
data_votos = pd.read_excel(
    open("camara.xlsx", "rb"),
    sheet_name="Votos"
)
data_curules = pd.read_excel(
    open("camara.xlsx", "rb"),
    sheet_name="Curules"
)
# Filtrando datos
votos = data_votos.loc[data_votos['Departamento'] == circunscripcion]
curules = data_curules.loc[data_curules['Departamento'] == circunscripcion]
# Limpiando datos votación escrutinio final 2018
nombre_listas = votos.to_dict(orient='split')['columns'][1:-1]
votos_listas = votos.to_dict(orient='split')['data'][0][1:-1]
votos_blancos = votos.to_dict(orient='split')['data'][0][-1]
curules_camara_ordinaria = curules.to_dict(orient='split')['data'][0][-1]
curules_camara_especial = 10
numero_listas = range(len(votos_listas))


In [4]:
# Cálculo de credenciales senado escrutinio final 2018
suma_listas = sum(votos_listas)
votos_validos = suma_listas + votos_blancos
umbral_old = votos_validos/(2*curules_camara_ordinaria)
personeria = round(votos_validos*(3/100))
umbral = max(umbral_old, personeria)
cifra_repartidora = repartidora(votos_listas, curules_camara_ordinaria)
proporciones, restos = cocientes(votos_listas, curules_camara_ordinaria)
# Distribución de curules
curules_proporcional = distribucion_proporcional(votos_listas, curules_camara_ordinaria)
curules_promedio = distribucion_promedio(votos_listas, cifra_repartidora, umbral)
curules_resto = distribucion_resto(votos_listas, curules_camara_ordinaria)
# Acumulado de curules
suma_proporcional = sum(curules_proporcional)
suma_promedio = sum(curules_promedio)
suma_resto = sum(curules_resto)
umbral

47520.6

In [5]:
print(f"*"*30)
print(f"Validos      : {votos_validos:15,.0f}")
print(f"Umbral       : {umbral:15,.0f}")
print(f"Repartidora  : {cifra_repartidora:15,.0f}")
print(f"*"*30)
print(f"-"*69)
print(f"|NUM | {'NOMBRE'.ljust(14)}| {'VOTOS'.ljust(10)}|D.PROP |D.PROM |D.REST |{'RESTO'.center(15)} |")
print(f"-"*69)
for item in numero_listas:
    print(
        f"|{item+1:3.0f} | {nombre_listas[item].ljust(14)}|{votos_listas[item]:10,.0f}",
        f"|{curules_proporcional[item]:6.0f} |{curules_promedio[item]:6.0f}",
        f"|{curules_resto[item]:6.0f} |{restos[item]:15,.0f} |")
print(f"-"*69)
print(
    f"| TOTALES    |{suma_listas:12,.0f} |{suma_proporcional:6.0f} |{suma_promedio:6.0f} |{suma_resto:6.0f} |")
print(f"-"*52)


******************************
Validos      :         475,206
Umbral       :          47,521
Repartidora  :          46,913
******************************
---------------------------------------------------------------------
|NUM | NOMBRE        | VOTOS     |D.PROP |D.PROM |D.REST |     RESTO      |
---------------------------------------------------------------------
|  1 | Liberal       |    75,888 |     0 |     1 |     1 |        379,440 |
|  2 | Conservador   |   140,738 |     1 |     2 |     1 |        256,065 |
|  3 | OC            |     1,232 |     0 |     0 |     0 |          6,160 |
|  4 | Radical       |    67,140 |     0 |     1 |     1 |        335,700 |
|  5 | Mira          |    10,969 |     0 |     0 |     0 |         54,845 |
|  6 | Unidad        |    68,699 |     0 |     1 |     1 |        343,495 |
|  7 | Polo          |    60,504 |     0 |     1 |     1 |        302,520 |
|  8 | Democratico   |    15,187 |     0 |     0 |     0 |         75,935 |
|  9 | Somos         

---
## Mas Recursos

- [Métodos del promedio mayor](https://es.wikipedia.org/wiki/M%C3%A9todo_de_promedios_mayores) (Wikipedia)
- [Método d'Hondt](https://es.wikipedia.org/wiki/Sistema_D%27Hondt) (Wikipedia)
- [Método Webster](https://es.wikipedia.org/wiki/M%C3%A9todo_Sainte-Lagu%C3%AB) (Wikipedia)
- [Métodos del resto mayor](https://es.wikipedia.org/wiki/M%C3%A9todo_del_resto_mayor) (Wikipedia)